In [1]:
import pyodbc
import pandas as pd

In [2]:
# Define connection details
server = 'EFGSVR050\\NOVY'
database = 'NovyDatabase'

In [3]:
# Create a connection string
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database}')

In [4]:
# SQL query to select the data from the NovyDB table for 2024
query = """
SELECT 
    DATEPART(YEAR, [Date]) AS Year,
    DATEPART(MONTH, [Date]) AS Month,
    CONCAT(System, ' - ', Channel) AS ChannelSystem, -- Combine Channel and System
    SalesOffice,
    ItemID,
    SUM(Revenue) AS Revenue
FROM [NovyDatabase].[dbo].[NovyDB] -- Use your actual table name
WHERE [Date] IS NOT NULL 
    AND DATEPART(YEAR, [Date]) <= 2023
GROUP BY 
    DATEPART(YEAR, [Date]), 
    DATEPART(MONTH, [Date]), 
    Channel, 
    System, -- Include System in GROUP BY since it is part of the concatenation
    SalesOffice, 
    ItemID
ORDER BY Year, Month, ChannelSystem, SalesOffice, ItemID;

"""

In [5]:
# Execute the query and fetch data into a pandas DataFrame
data = pd.read_sql(query, conn)

C:\Users\aya.tarek\AppData\Local\Temp\2\ipykernel_39208\1709954960.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn)


In [6]:
# Close the connection
conn.close()

In [7]:
# Display the first few rows of the fetched data
data.head()

,Year,Month,ChannelSystem,SalesOffice,ItemID,Revenue
0,2021,12,SAP - Agent Confectionery,A013,96228,555.0
1,2021,12,SAP - Agent Confectionery,A013,96229,555.0
2,2021,12,SAP - Agent Confectionery,A013,96230,2220.0
3,2021,12,SAP - Agent Confectionery,A013,96231,1110.0
4,2021,12,SAP - Agent Confectionery,A013,96232,2220.0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71083 entries, 0 to 71082
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           71083 non-null  int64  
 1   Month          71083 non-null  int64  
 2   ChannelSystem  71083 non-null  object 
 3   SalesOffice    71083 non-null  object 
 4   ItemID         71083 non-null  object 
 5   Revenue        71083 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 3.3+ MB


In [9]:
data.isna().sum()

Year             0
Month            0
ChannelSystem    0
SalesOffice      0
ItemID           0
Revenue          0
dtype: int64

In [10]:
data.describe().round(2)

,Year,Month,Revenue
count,71083.00,71083.00,71083.00
mean,2022.54,6.87,27155.31
std,0.50,3.34,68862.95
min,2021.00,1.00,-182400.00
25%,2022.00,4.00,1320.00
50%,2023.00,7.00,6960.00
75%,2023.00,10.00,25740.00
max,2023.00,12.00,3264739.17


In [12]:
# Step 2: Create Lag Features
data.sort_values(by=['Year', 'Month', 'ChannelSystem', 'SalesOffice', 'ItemID'], inplace=True)
data['Revenue_Lag_1'] = data.groupby(['ChannelSystem', 'SalesOffice', 'ItemID'])['Revenue'].shift(1)
data['Revenue_Lag_2'] = data.groupby(['ChannelSystem', 'SalesOffice', 'ItemID'])['Revenue'].shift(2)
data['Revenue_Lag_12'] = data.groupby(['ChannelSystem', 'SalesOffice', 'ItemID'])['Revenue'].shift(12)

In [13]:
# Step 3: Set Target for 2024
achieved_revenue_2023 = data[data['Year'] == 2023]['Revenue'].sum()
overall_target_2024 = achieved_revenue_2023 * 1.5

In [14]:
# Step 4: Prepare Data for Training
features = ['Revenue_Lag_1', 'Revenue_Lag_2', 'Revenue_Lag_12']
target = 'Revenue'

train_data = data[data['Year'] < 2023]
X_train = train_data[features]
y_train = train_data[target]

In [15]:
# Create a template for future predictions
future_template = data[data['Year'] == 2023].copy()
future_template['Year'] = 2024  # Set Year to 2024
future_template['Revenue'] = 0  # Placeholder for predictions
X_future = future_template[features]

In [16]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
# Step 5: Train the Model
model = LGBMRegressor(random_state=42)
model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001285 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 32832, number of used features: 2
[LightGBM] [Info] Start training from score 21988.793827


LGBMRegressor(random_state=42)

In [17]:
# Step 6: Predict for 2024
future_template['Predicted_Revenue'] = model.predict(X_future)

In [18]:
# Step 7: Scale Predictions to Meet Overall Target
total_predicted_revenue_2024 = future_template['Predicted_Revenue'].sum()
future_template['Scaled_Revenue'] = future_template['Predicted_Revenue'] / total_predicted_revenue_2024 * overall_target_2024


In [19]:
# Step 8: Output Results
output_columns = ['Year', 'Month', 'ChannelSystem', 'SalesOffice', 'ItemID', 'Scaled_Revenue']
scaled_predictions = future_template[output_columns]

print(f"Total Predicted Revenue for 2024 (Scaled): {scaled_predictions['Scaled_Revenue'].sum()}")
print(scaled_predictions.head(10))  # Display first 10 rows

Total Predicted Revenue for 2024 (Scaled): 1812517494.8920858
       Year  Month ChannelSystem SalesOffice ItemID  Scaled_Revenue
32832  2024      1   Agent - RTL    11010301  96237    65611.558979
32833  2024      1   Agent - RTL    11010301  96276    15936.567701
32834  2024      1   Agent - RTL    11010301  96284    11879.653300
32835  2024      1   Agent - RTL    11010301  96286    26325.139458
32836  2024      1   Agent - RTL    11010301  96291    22868.075099
32837  2024      1   Agent - RTL    11010301  96300    24276.574427
32838  2024      1   Agent - RTL    11010301  96301    72319.519356
32839  2024      1   Agent - RTL    11010301  96303    19930.829264
32840  2024      1   Agent - RTL    11010301  96311    15435.420635
32841  2024      1   Agent - RTL    11010301  96312    15350.808782


In [20]:
# Establish connection to SQL Server
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database}')
cursor = conn.cursor()

In [21]:
create_table_query = """
CREATE TABLE SalesPrediction (
    Year INT,
    Month INT,
    ChannelSystem NVARCHAR(100),
    SalesOffice NVARCHAR(100),
    ItemID NVARCHAR(50),
    Scaled_Revenue FLOAT
);
"""

In [22]:
# Execute the query to create the table
cursor.execute(create_table_query)
conn.commit()

In [23]:
# Close the connection
cursor.close()
conn.close()

In [24]:
# Re-establish connection to SQL Server
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database}')
cursor = conn.cursor()

# Insert the monthly_data into the MonthlySales table
for index, row in scaled_predictions.iterrows():
    insert_query = """
    INSERT INTO [NovyDatabase].[dbo].[SalesPrediction] 
    (Year, Month, ChannelSystem, SalesOffice, ItemID, Scaled_Revenue)
    VALUES (?, ?, ?, ?, ?, ?)
    """
    
    # Execute the query with row data
    cursor.execute(insert_query, 
                   row['Year'], 
                   row['Month'], 
                   row['ChannelSystem'], 
                   row['SalesOffice'], 
                   row['ItemID'], 
                   row['Scaled_Revenue'])

# Commit the transaction
conn.commit()

# Close the connection
cursor.close()
conn.close()

print("Data successfully inserted into MonthlySales table.")

Data successfully inserted into MonthlySales table.
